## API AlphaVantage - Intradiario

#### Importar las librerías necesarias

In [6]:
import requests
from decouple import config
import json
import pandas as pd
import psycopg2

#### Leer la contraseña y el token desde la variable de entorno

In [7]:
my_host = config('DATABASE_HOST')
port = config('DATABASE_PORT')
password = config('DATABASE_PASSWORD')
database_name = config('DATABASE_NAME')
user = config('DATABASE_USER')
token = config('ALPHAVANTAGE_TOKEN')

#### Definir la función para obtener datos de AlphaVantage

In [8]:
def obtener_datos_acciones(symbols, token):
    dataframes = []
    for symbol in symbols:
        url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={token}'
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if 'Time Series (Daily)' in data:
                time_series_data = data['Time Series (Daily)']
                df = pd.DataFrame(time_series_data).T
                df['Fecha'] = df.index
                df['symbol'] = symbol
                df = df[['Fecha', '1. open', '2. high', '3. low', '4. close', '5. volume', 'symbol']]
                df.columns = ['Fecha', 'Apertura', 'Maximo', 'Minimo', 'Cierre', 'Volumen', 'Symbol']
                dataframes.append(df)
            else:
                print(f"No se encontraron datos para {symbol}.")
        else:
            print(f"No se pudo obtener datos para {symbol}. Código de estado: {response.status_code}")

    if dataframes:
        return pd.concat(dataframes)
    else:
        return None

# Cargar token de Alphavantage desde el archivo .env
token = config('ALPHAVANTAGE_TOKEN')

# Símbolos de las acciones que deseas obtener
symbols = ['AMZN', 'MELI', 'AAPL', 'GOOGL', 'PINS']

# Obtener datos de Alphavantage y mostrar en un DataFrame
df = obtener_datos_acciones(symbols, token)
if df is not None:
    print(df)
else:
    print("No se pudieron obtener datos para ninguna acción.")

                 Fecha  Apertura    Maximo    Minimo    Cierre   Volumen  \
2024-03-28  2024-03-28  180.1700  181.7000  179.2600  180.3800  38051588   
2024-03-27  2024-03-27  179.8800  180.0000  177.3099  179.8300  33272551   
2024-03-26  2024-03-26  180.1500  180.4500  177.9500  178.3000  29658982   
2024-03-25  2024-03-25  178.0100  180.9900  177.2400  179.7100  29815464   
2024-03-22  2024-03-22  177.7520  179.2550  176.7500  178.8700  27995378   
...                ...       ...       ...       ...       ...       ...   
2023-11-09  2023-11-09   31.5709   31.8700   30.9550   31.2600   9249145   
2023-11-08  2023-11-08   30.7200   31.6175   30.6500   31.3700   7882682   
2023-11-07  2023-11-07   30.9000   31.3600   30.6500   30.9500  13478954   
2023-11-06  2023-11-06   31.0500   31.1400   30.3200   30.9200   9929900   
2023-11-03  2023-11-03   30.9500   31.3600   30.5050   30.9400  14262650   

           Symbol  
2024-03-28   AMZN  
2024-03-27   AMZN  
2024-03-26   AMZN  
2024-03

#### Conectarse con Amazon Redshift y creación de tabla (si no existe)

In [9]:

#Conectar a la base de datos
try:
    conn = psycopg2.connect(
        host=my_host, 
        port=port, 
        database=database_name, 
        user=user, 
        password=password
    )
    print("Conexión a la base de datos exitosa")

    # Crear tabla
    try:
        cur = conn.cursor()
        cur.execute("""
        CREATE TABLE IF NOT EXISTS hvzambrana_coderhouse.alphavantage (
            simbolo VARCHAR(10) DISTKEY NOT NULL,
            fecha TIMESTAMP SORTKEY NOT NULL,
            apertura FLOAT(15) NULL,
            maximo FLOAT(15) NULL,
            minimo FLOAT(15) NULL,
            cierre FLOAT(15) NULL,
            volumen FLOAT(15) NULL
        );
        """)
        conn.commit()
        print("Tabla creada exitosamente")
    except Exception as e:
            print("Error al crear la tabla")
            print(e)

except Exception as e:
    print("Error al crear la conexión a la base de datos")
    print(e)
finally:
    if conn is not None:
        conn.close()
        print("Conexión cerrada correctamente")

Conexión a la base de datos exitosa
Tabla creada exitosamente
Conexión cerrada correctamente
